In [8]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier


readdata = pd.read_csv("/Users/academic/Desktop/TheGrandTour/data/ParticleData.txt", sep=" ", header=None);
data = np.array(readdata);
data = np.delete(data, 0, 0)
data = data.astype(float)
data = np.swapaxes(data,0,1)


# Need to seperate the classification dimension:
classIndex = np.shape(data)[0] - 1
classification = data[classIndex]
data = np.delete(data, classIndex, axis=0)


# make list of colours for each number:

# make list of colours for each number:
colour = np.chararray((len(classification)),unicode = True)
for i in range(len(classification)):
    if classification[i] == 1:
        colour[i] = 'r'
    elif classification[i] == 0:
        classification[i] = 2
        colour[i] = 'b'


for i in range(0, np.shape(data)[0]):
    data[i,:] = data[i,:] - np.min(data[i,:])
    data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1
    

def getAlpha(d,stepSize):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.

    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)
            
    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d

    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane

    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane

    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    Alpha = getAlpha(d)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData

def Transform(data,alpha):
    d = np.shape(data)[0]
    theta = getAngles(alpha,d)
    b = BetaFn(d,theta)
    a = np.matmul(b,data)
    return a
    
def SVM(a):
    xData = a[0]
    yData = a[1]
    nData = np.vstack((xData, yData)).T
    
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    score = clf.score(nData, classification)
    return score

def ResolveGrandTour(data, nSteps,stepSize,alphaMax):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    TData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    score = np.zeros((nSteps))
    Alpha = getAlpha(d,stepSize)
    alphaStart = alphaMax - Alpha.dot(nSteps/2)
    
    for t in range(0, nSteps):
        alpha = alphaStart + Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        #w = SVM(a)
        w = 1
        print(w)
        TData[t,:,:] = a
        #tBeta[t,:,:] = b
        score[t] = w
        
    iMax = np.argmax(score)
    alphaMax = alpha = alphaStart + Alpha.dot(iMax)
    return TData, score, alphaMax

def HighResGrandTour(resolution,initialStepSize,nSteps):
    d = np.shape(data)[0]
    nPoints = np.shape(data)[1]
    stepSize = 0.1   
    alphaMax = getAlpha(d,0)
    tData = np.zeros((0,d,nPoints))
    score = np.zeros((resolution,nSteps))
    for i in range(0,resolution):
        print(i)
        TData, Score, alphaMax = ResolveGrandTour(data, nSteps,stepSize,alphaMax)
        stepSize = stepSize/(10**i)
        tData = np.append(tData,TData,axis = 0)
        score[i] = Score
    return tData, score

In [9]:
tData, score = HighResGrandTour(3,0.1,10)

0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1


In [10]:
import sys
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore

if __name__ == '__main__':
    
    app = 0
    app = QtGui.QApplication([])
    win = pg.GraphicsWindow(title = "GrandTour with SVM")
    win.resize(1200,800)
    win.setWindowTitle('GrandTour with SVM')
    pg.setConfigOptions(antialias=True) #dunno what that means


    ## create four areas to add plots
    w1 = win.addPlot()



    ## There are a few different ways we can draw scatter plots; each is optimized for different types of data:


    ## 1) All spots identical and transform-invariant (top-left plot). 
    ## In this case we can get a huge performance boost by pre-rendering the spot 
    ## image and just drawing that image repeatedly.
    nPoints = np.shape(tData)[2]
    s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
    j = 0
    pos = np.zeros((nPoints,2))
    pos[:,0] = tData[0,0,:]
    pos[:,1] = tData[0,1,:]
    s1 = pg.ScatterPlotItem(pos=pos, brush = colour, size=0.05, pxMode=False,pen=pg.mkPen(None))
    w1.addItem(s1)


    w2 = win.addPlot(title="SVM")
    w2.setXRange(0,100,padding = None)
    w2.setYRange(0,1,padding = None)
    curve = w2.plot(pen='r')
    res = 0

    def update():
        global j, res
        j += 1
        if j > np.shape(tData)[0]-1:
            j = 0
            res = -1
        pos[:,0] = tData[j,0,:]
        pos[:,1] = tData[j,1,:]
        s1.setData(pos = pos,brush = colour)
        
        if j%nSteps == 0: 
            res = res + 1
        
        curve.setData(score[res,0:j%nSteps])
        w2.setXRange(0,nSteps,padding = None)

    t = QtCore.QTimer()
    t.timeout.connect(update)
    t.start(50)

## 2) Spots are transform-invariant, but not identical (top-right plot). 
## In this case, drawing is almsot as fast as 1), but there is more startup 
## overhead and memory usage since each spot generates its own pre-rendered 
## image.
    sys.exit(app.exec_())
    


/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'nSteps' is not defined

NameError: name 'nSteps' is not defined

NameError: name 'nSteps' is not defined

NameError: name 'nSteps' is not defined

...

NameError: name 'nSteps' is not defined

NameError: name 'nSteps' is not defined

NameError: name 'nSteps' is not defined

SystemExit: 0

/Users/karanmukhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
np.shape(data)[0]


13